In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from IPython.display import display


%matplotlib inline

In [2]:
#read the data and drop redundant coloumn
data=pd.read_csv("final_dataset.csv")




In [3]:
display(data.head())


,Unnamed: 0,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTGS,ATGS,HTGC,...,HTLossStreak5,ATWinStreak3,ATWinStreak5,ATLossStreak3,ATLossStreak5,HTGD,ATGD,DiffPts,DiffFormPts,DiffLP
0,0,2000-08-19,Charlton,Man City,4,0,H,0,0,0,...,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0
1,1,2000-08-19,Chelsea,West Ham,4,2,H,0,0,0,...,0,0,0,0,0,0.0,0.0,0.0,0.0,-4.0
2,2,2000-08-19,Coventry,Middlesbrough,1,3,NH,0,0,0,...,0,0,0,0,0,0.0,0.0,0.0,0.0,2.0
3,3,2000-08-19,Derby,Southampton,2,2,NH,0,0,0,...,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0
4,4,2000-08-19,Leeds,Everton,2,0,H,0,0,0,...,0,0,0,0,0,0.0,0.0,0.0,0.0,-10.0


In [4]:
data = data[data.MW > 3]

data.drop(['Unnamed: 0','HomeTeam', 'AwayTeam', 'Date', 'MW', 'HTFormPtsStr', 'ATFormPtsStr', 'FTHG', 'FTAG',
           'HTGS', 'ATGS', 'HTGC', 'ATGC','HomeTeamLP', 'AwayTeamLP','DiffPts','HTFormPts','ATFormPts',
           'HM4','HM5','AM4','AM5','HTLossStreak5','ATLossStreak5','HTWinStreak5','ATWinStreak5',
           'HTWinStreak3','HTLossStreak3','ATWinStreak3','ATLossStreak3'],1, inplace=True)

In [6]:
display(data.head())

,FTR,HTP,ATP,HM1,HM2,HM3,AM1,AM2,AM3,HTGD,ATGD,DiffFormPts,DiffLP
30,H,1.25,1.00,D,D,W,D,W,L,0.50,0.25,0.25,-16.0
31,NH,0.75,0.25,L,L,W,D,L,L,-0.50,-0.75,0.50,-2.0
32,H,1.00,1.00,L,D,W,D,W,L,0.00,0.25,0.00,-3.0
33,NH,0.75,0.50,L,L,W,D,L,D,-0.25,-0.25,0.25,3.0
34,NH,1.00,1.50,D,L,W,W,W,L,0.00,0.75,-0.50,3.0


In [5]:
# Total number of matches
n_matches = data.shape[0]

# Calculate number of features.
n_features = data.shape[1] - 1

# Calculate matches won by home team.
n_homewins = len(data[data.FTR == 'H'])

# Calculate win rate for home team.
win_rate = (float(n_homewins) / (n_matches)) * 100

# Print the results
print ("Total number of matches: {}".format(n_matches))
print ("Number of features: {}".format(n_features))
print ("Number of matches won by home team: {}".format(n_homewins))
print ("Win rate of home team: {:.2f}%".format(win_rate))

Total number of matches: 5600
Number of features: 12
Number of matches won by home team: 2603
Win rate of home team: 46.48%


In [6]:
# Separate into feature set and target variable
X_all = data.drop(['FTR'],1)
y_all = data['FTR']

# Standardising the data.
from sklearn.preprocessing import scale


cols = [['HTGD','ATGD','HTP','ATP','DiffLP']]
for col in cols:
    X_all[col] = scale(X_all[col])
    

In [7]:
display(X_all.head())

,HTP,ATP,HM1,HM2,HM3,AM1,AM2,AM3,HTGD,ATGD,DiffFormPts,DiffLP
30,-0.043829,-0.611968,D,D,W,D,W,L,0.753719,0.355995,0.25,-1.989216
31,-1.120644,-2.238746,L,L,W,D,L,L,-0.737082,-1.138834,0.50,-0.248963
32,-0.582236,-0.611968,L,D,W,D,W,L,0.008318,0.355995,0.00,-0.373267
33,-1.120644,-1.696487,L,L,W,D,L,D,-0.364382,-0.391419,0.25,0.372556
34,-0.582236,0.472551,D,L,W,W,W,L,0.008318,1.103409,-0.50,0.372556


In [8]:
display(y_all.head())

30     H
31    NH
32     H
33    NH
34    NH
Name: FTR, dtype: object

In [9]:
X_all.HM1 = X_all.HM1.astype('str')
X_all.HM2 = X_all.HM2.astype('str')
X_all.HM3 = X_all.HM3.astype('str')
X_all.AM1 = X_all.AM1.astype('str')
X_all.AM2 = X_all.AM2.astype('str')
X_all.AM3 = X_all.AM3.astype('str')

In [10]:
def preprocess_features(X):
    ''' Preprocesses the football data and converts catagorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix = col)
                    
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print ("Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns)))

Processed feature columns (24 total features):
['HTP', 'ATP', 'HM1_D', 'HM1_L', 'HM1_W', 'HM2_D', 'HM2_L', 'HM2_W', 'HM3_D', 'HM3_L', 'HM3_W', 'AM1_D', 'AM1_L', 'AM1_W', 'AM2_D', 'AM2_L', 'AM2_W', 'AM3_D', 'AM3_L', 'AM3_W', 'HTGD', 'ATGD', 'DiffFormPts', 'DiffLP']


In [11]:
display(X_all.head())

,HTP,ATP,HM1_D,HM1_L,HM1_W,HM2_D,HM2_L,HM2_W,HM3_D,HM3_L,...,AM2_D,AM2_L,AM2_W,AM3_D,AM3_L,AM3_W,HTGD,ATGD,DiffFormPts,DiffLP
30,-0.043829,-0.611968,1,0,0,1,0,0,0,0,...,0,0,1,0,1,0,0.753719,0.355995,0.25,-1.989216
31,-1.120644,-2.238746,0,1,0,0,1,0,0,0,...,0,1,0,0,1,0,-0.737082,-1.138834,0.50,-0.248963
32,-0.582236,-0.611968,0,1,0,1,0,0,0,0,...,0,0,1,0,1,0,0.008318,0.355995,0.00,-0.373267
33,-1.120644,-1.696487,0,1,0,0,1,0,0,0,...,0,1,0,1,0,0,-0.364382,-0.391419,0.25,0.372556
34,-0.582236,0.472551,1,0,0,0,1,0,0,0,...,0,0,1,0,1,0,0.008318,1.103409,-0.50,0.372556


In [12]:
from sklearn.cross_validation import train_test_split

# Shuffle and split the dataset into training and testing set.
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, 
                                                    test_size = 50,
                                                    random_state = 2,
                                                    stratify = y_all)

/home/dewaniadi/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [13]:
display(X_train.head())

,HTP,ATP,HM1_D,HM1_L,HM1_W,HM2_D,HM2_L,HM2_W,HM3_D,HM3_L,...,AM2_D,AM2_L,AM2_W,AM3_D,AM3_L,AM3_W,HTGD,ATGD,DiffFormPts,DiffLP
2025,1.074403,0.055428,0,0,1,0,0,1,1,0,...,0,1,0,0,0,1,1.040411,-0.017712,0.076923,-1.616304
1343,-0.992452,-0.611968,0,1,0,0,1,0,1,0,...,0,1,0,0,1,0,-1.624464,-0.515989,0.095238,0.621164
4059,1.737058,1.056523,0,1,0,0,0,1,0,0,...,0,1,0,0,0,1,1.785812,0.959676,0.076923,-0.373267
82,-0.103652,-1.093976,0,0,1,1,0,0,0,0,...,0,0,1,1,0,0,0.173963,-0.183804,0.111111,1.118379
1697,0.255287,-0.250462,1,0,0,0,1,0,1,0,...,0,1,0,1,0,0,0.505252,-0.349896,0.000000,-0.124659


In [14]:
#for measuring training time
from time import time 

from sklearn.metrics import f1_score

def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print ("Trained model in {:.4f} seconds".format(end - start))

    

In [15]:
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    
    end = time()
    # Print and return results
    print ("Made predictions in {:.4f} seconds.".format(end - start))
    
    return f1_score(target, y_pred, pos_label='H'), sum(target == y_pred) / float(len(y_pred))

In [16]:
def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print ("Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train)))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    f1, acc = predict_labels(clf, X_train, y_train)
    print (f1, acc)
    print ("F1 score and accuracy score for training set: {:.4f} , {:.4f}.".format(f1 , acc))
    
    f1, acc = predict_labels(clf, X_test, y_test)
    print ("F1 score and accuracy score for test set: {:.4f} , {:.4f}.".format(f1 , acc)) 

In [17]:
clf_A = LogisticRegression(random_state = 42)
clf_B = SVC(random_state = 912, kernel='rbf')
clf_C = xgb.XGBClassifier(seed = 82)

train_predict(clf_A, X_train, y_train, X_test, y_test)
print ('')
train_predict(clf_B, X_train, y_train, X_test, y_test)
print ('')
train_predict(clf_C, X_train, y_train, X_test, y_test)
print ('')

Training a LogisticRegression using a training set size of 5550. . .
Trained model in 0.0345 seconds
Made predictions in 0.0019 seconds.
0.6215610352557571 0.6654054054054054
F1 score and accuracy score for training set: 0.6216 , 0.6654.
Made predictions in 0.0005 seconds.
F1 score and accuracy score for test set: 0.6957 , 0.7200.

Training a SVC using a training set size of 5550. . .
Trained model in 1.9781 seconds
Made predictions in 1.1772 seconds.
0.6204535729567822 0.6803603603603604
F1 score and accuracy score for training set: 0.6205 , 0.6804.
Made predictions in 0.0109 seconds.
F1 score and accuracy score for test set: 0.6818 , 0.7200.

Training a XGBClassifier using a training set size of 5550. . .
Trained model in 0.4970 seconds
Made predictions in 0.0253 seconds.
0.6521471132114238 0.694954954954955
F1 score and accuracy score for training set: 0.6521 , 0.6950.
Made predictions in 0.0022 seconds.
F1 score and accuracy score for test set: 0.7451 , 0.7400.



/home/dewaniadi/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/dewaniadi/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [18]:
clf_A.predict(X_test)

array(['H', 'H', 'H', 'NH', 'NH', 'H', 'NH', 'H', 'H', 'H', 'NH', 'NH',
       'H', 'NH', 'H', 'NH', 'H', 'H', 'H', 'NH', 'NH', 'H', 'NH', 'NH',
       'NH', 'NH', 'H', 'NH', 'NH', 'H', 'H', 'H', 'NH', 'H', 'NH', 'H',
       'H', 'NH', 'NH', 'NH', 'NH', 'H', 'NH', 'NH', 'NH', 'NH', 'NH',
       'H', 'NH', 'H'], dtype=object)

In [20]:
clf_B.predict(X_test)


array(['H', 'H', 'H', 'NH', 'NH', 'H', 'NH', 'H', 'H', 'H', 'NH', 'NH',
       'H', 'NH', 'H', 'NH', 'H', 'H', 'H', 'NH', 'NH', 'H', 'NH', 'NH',
       'NH', 'NH', 'H', 'NH', 'NH', 'H', 'NH', 'NH', 'NH', 'H', 'NH', 'H',
       'H', 'NH', 'NH', 'NH', 'NH', 'H', 'NH', 'NH', 'NH', 'NH', 'NH',
       'H', 'NH', 'H'], dtype=object)

In [ ]:
clf_C.predict(X_test)

In [19]:
clf_C.predict(X_test)

/home/dewaniadi/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array(['H', 'H', 'H', 'NH', 'H', 'H', 'NH', 'H', 'H', 'H', 'NH', 'NH',
       'H', 'H', 'H', 'H', 'H', 'H', 'H', 'NH', 'NH', 'H', 'NH', 'NH',
       'NH', 'H', 'H', 'NH', 'NH', 'H', 'H', 'H', 'NH', 'H', 'NH', 'H',
       'H', 'H', 'H', 'NH', 'NH', 'NH', 'NH', 'NH', 'NH', 'NH', 'NH', 'H',
       'NH', 'H'], dtype=object)

,HTP,ATP,HM1_D,HM1_L,HM1_W,HM2_D,HM2_L,HM2_W,HM3_D,HM3_L,...,AM2_D,AM2_L,AM2_W,AM3_D,AM3_L,AM3_W,HTGD,ATGD,DiffFormPts,DiffLP
5879,0.733871,-0.732470,0,0,1,0,1,0,1,0,...,1,0,0,0,1,0,1.167830,-0.765127,0.166667,-1.616304
1011,0.743075,-0.194845,0,0,1,0,0,1,0,1,...,1,0,0,0,1,0,0.122995,0.039781,0.038462,1.242683
6022,1.310348,0.702601,1,0,0,0,0,1,0,0,...,0,0,1,0,0,1,1.453943,0.389968,-0.030303,0.123949
5009,-0.274575,1.866933,0,0,1,0,0,1,1,0,...,0,0,1,0,0,1,0.221290,1.263569,-1.000000,0.496860
4717,0.225375,-0.205273,0,0,1,0,1,0,0,1,...,0,1,0,1,0,0,0.194668,0.075714,-0.062500,1.242683
2557,1.263733,0.550017,0,0,1,0,0,1,1,0,...,0,0,1,0,0,1,1.445876,0.355995,0.000000,-1.740608
1503,1.396776,1.615693,0,1,0,0,1,0,0,0,...,0,1,0,1,0,0,1.096200,1.436715,0.027027,-0.373267
5734,2.109802,-1.154227,0,0,1,0,0,1,0,0,...,1,0,0,1,0,0,2.989920,-0.017712,1.500000,-1.989216
4907,-0.274575,-1.293666,0,1,0,0,1,0,1,0,...,1,0,0,0,1,0,-0.332436,-1.213575,0.085714,-1.119089
291,0.422791,-0.467365,0,0,1,0,0,1,0,0,...,0,1,0,0,0,1,0.008318,-0.615644,0.333333,0.248252
